<a href="https://colab.research.google.com/github/KorneliuszDyszczynski/ColorizingPhotosNeuralNetwork/blob/master/ColorizingImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import keras
import numpy as np
import keras.preprocessing.image
from keras.preprocessing.image import img_to_array, load_img
from skimage.color import rgb2lab
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from kaggle.api.kaggle_api_extended import KaggleApi
from PIL import Image
import os, sys

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
api = KaggleApi()
api.authenticate()
api.dataset_download_files('ashwingupta3012/human-faces')

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!unzip human-faces.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
  inflating: Humans/1 (2910).jpg     
  inflating: Humans/1 (2911).jpg     
  inflating: Humans/1 (2912).jpg     
  inflating: Humans/1 (2913).jpg     
  inflating: Humans/1 (2914).jpg     
  inflating: Humans/1 (2915).jpg     
  inflating: Humans/1 (2916).jpg     
  inflating: Humans/1 (2917).jpg     
  inflating: Humans/1 (2918).jpg     
  inflating: Humans/1 (2919).jpg     
  inflating: Humans/1 (292).jpg      
  inflating: Humans/1 (2920).jpg     
  inflating: Humans/1 (2921).jpg     
  inflating: Humans/1 (2922).jpg     
  inflating: Humans/1 (2923).jpg     
  inflating: Humans/1 (2924).jpg     
  inflating: Humans/1 (2925).jpg     
  inflating: Humans/1 (2926).jpg     
  inflating: Humans/1 (2927).jpg     
  inflating: Humans/1 (2928).jpg     
  inflating: Humans/1 (2929).jpg     
  inflating: Humans/1 (293).jpg      
  inflating: Humans/1 (2930).jpg     
  inflating: Humans/1 (2931).jpg     
  inflating: Humans/1 (

In [89]:
os.chdir('/content/Humans/') 
path='/content/Humans/'
path2='/content/Humans2/'
path3='/content/'
COUNT =0
w = 200
h = 200
stop=20;

# for f in os.listdir():
#     f_name, f_ext = os.path.splitext(f)
#     f_name = "face" + str(COUNT) 
#     COUNT = COUNT + 1
#     new_name = '{}{}'.format(f_name, f_ext)
#     os.rename(f, new_name)

for f in os.listdir():
    COUNT = COUNT + 1
    if(COUNT<stop):
      im = Image.open(path+f)
      im = im.resize((w, h))
      im = im.convert('RGB')
      im.save(path2+"resized"+f)

os.chdir('/content/Humans2/') 

images = [Image.open(f) for f in os.listdir()]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for im in images:
  new_im.paste(im, (x_offset,0))
  x_offset += im.size[0]

new_im.save(path3+'test.jpg')


image = img_to_array(load_img(path3+'test.jpg'))
image = np.array(image, dtype=float)


In [82]:
!rm -r ../Humans2/*

In [91]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y = Y / 128
witdthvar=(stop-1)*200;
X = X.reshape(1, 200, witdthvar, 1)
Y = Y.reshape(1, 200, witdthvar, 2)

In [92]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.compile(optimizer='rmsprop',loss='mse')

In [93]:
model.fit(x=X, y=Y, batch_size=1, epochs=3000)
print(model.evaluate(X, Y, batch_size=1))

Epoch 1/3000
1/1 [==============================] - 3s 3s/step - loss: 0.0480
Epoch 2/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0146
Epoch 3/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0106
Epoch 4/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0064
Epoch 5/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0062
Epoch 6/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0060
Epoch 7/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0059
Epoch 8/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0058
Epoch 9/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0058
Epoch 10/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0057
Epoch 11/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0058
Epoch 12/3000
1/1 [==============================] - 1s 1s/step - loss: 0.0058
Epoch 13/3000
1/1 [==============================] - 1s 1s/st

KeyboardInterrupt: ignored